In [0]:
! git clone https://github.com/tyiannak/pyAudioAnalysis.git

[source article](https://walczak.org/2019/02/automatic-splitting-audio-files-silence-python/)

[source github](https://github.com/tyiannak/pyAudioAnalysis)

In [0]:
! pip install -r /content/pyAudioAnalysis/requirements.txt

In [0]:
import glob, os
src_dir = "" # source directory path
for file in sorted(glob.glob(f"{src_dir}/*.wav")):
    print("#"*40)
    print(file)
    src_code = "/content/pyAudioAnalysis/pyAudioAnalysis/audioAnalysis.py" # running using the sourcecode 
    os.system(f"python {src_code} silenceRemoval -i " + file + " --smoothing 0.2 --weight 0.1")